<a href="https://colab.research.google.com/github/aditi2259/Automated-Information-Retrieval-Dashboard/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("peopledatalabssf/free-7-million-company-dataset")

print("Path to dataset files:", path)

100%|██████████| 278M/278M [00:03<00:00, 73.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/peopledatalabssf/free-7-million-company-dataset/versions/1


In [2]:
import pandas as pd
import os
import kagglehub

dataset_path = kagglehub.dataset_download("peopledatalabssf/free-7-million-company-dataset")

for filename in os.listdir(dataset_path):
    if filename.endswith(".csv"):
        dataset_file = os.path.join(dataset_path, filename)
        break
else:
    raise FileNotFoundError("No CSV file found in the dataset directory.")

# Load the dataset
df = pd.read_csv(dataset_file)


print("Dataset loaded successfully!")
print(df.head())

Dataset loaded successfully!
   Unnamed: 0                       name         domain  year founded  \
0     5872184                        ibm        ibm.com        1911.0   
1     4425416  tata consultancy services        tcs.com        1968.0   
2       21074                  accenture  accenture.com        1989.0   
3     2309813                    us army     goarmy.com        1800.0   
4     1558607                         ey         ey.com        1989.0   

                              industry size range  \
0  information technology and services     10001+   
1  information technology and services     10001+   
2  information technology and services     10001+   
3                             military     10001+   
4                           accounting     10001+   

                                 locality         country  \
0       new york, new york, united states   united states   
1              bombay, maharashtra, india           india   
2                 dublin, dubl

In [3]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.2 MB/s eta 0:00:00


In [4]:
import streamlit as st

In [6]:
st.title("Company Information Retrieval Dashboard")
st.header("Dataset Preview")

# Display dataset
st.write("Preview of the company dataset:")
st.dataframe(df)

selected_column = st.selectbox("Select the primary column for entities", df.columns)

query_template = st.text_input("Enter your query template (e.g., 'Find the headquarters of {entity}')")


2024-11-18 17:27:34.924 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:27:34.928 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:27:34.932 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:27:34.934 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:27:34.937 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:27:34.939 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:27:34.941 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:27:34.944 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [7]:
import requests

# using SerpAPI
def perform_web_search(entity):
    api_key = st.secrets["serpapi_key"]
    query = query_template.replace("{entity}", entity)
    params = {
        "q": query,
        "api_key": api_key,
        "engine": "google",
    }
    response = requests.get("https://serpapi.com/search", params=params)
    return response.json()

if st.button("Perform Web Search"):
    search_results = {}
    for entity in df[selected_column].dropna().unique()[:10]:
        search_results[entity] = perform_web_search(entity)
    st.success("Web search completed.")
    st.json(search_results)


2024-11-18 17:28:17.397 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:28:17.400 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:28:17.401 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:28:17.403 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:28:17.404 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [8]:
import openai

# LLM parsing function
def process_with_llm(entity, search_results):
    api_key = st.secrets["openai_api_key"]
    prompt = f"Extract the headquarters location for {entity} from the following search results: {search_results}"
    openai.api_key = api_key
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Extract relevant information."},
            {"role": "user", "content": prompt}
        ]
    )
    return response["choices"][0]["message"]["content"]

if st.button("Parse with LLM"):
    parsed_data = {}
    for entity, results in search_results.items():
        parsed_data[entity] = process_with_llm(entity, results)
    st.success("LLM processing completed.")
    st.json(parsed_data)


2024-11-18 17:28:32.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:28:32.754 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:28:32.759 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:28:32.762 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:28:32.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [9]:
if st.button("Parse with LLM"):
    parsed_data = {}
    for entity, results in search_results.items():
        parsed_data[entity] = process_with_llm(entity, results)
    st.success("LLM processing completed.")
    st.json(parsed_data)

2024-11-18 17:28:45.878 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:28:45.880 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:28:45.882 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:28:45.885 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:28:45.888 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [10]:
parsed_data = {}


In [11]:

if 'parsed_data' not in locals() or not parsed_data:
    st.error("Parsed data is empty or not available. Please complete the earlier steps.")
    st.stop()

# Convert parsed_data to DataFrame
parsed_df = pd.DataFrame.from_dict(parsed_data, orient="index", columns=["Extracted Info"])

st.write("Extracted Data:")
st.dataframe(parsed_df)

# Download button for exporting the data
csv_data = parsed_df.to_csv(index=True)
st.download_button(
    label="Download Extracted Data as CSV",
    data=csv_data,
    file_name="extracted_data.csv",
    mime="text/csv"
)


2024-11-18 17:29:42.502 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:29:42.504 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:29:42.506 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:29:42.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:29:42.516 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:29:42.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:29:42.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:29:42.523 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

False

In [12]:
# Perform Web Search Block
if st.button("Perform Web Search"):
    if not selected_column or not query_template:
        st.error("Please select a column and provide a query template.")
    else:
        try:

            search_results = {}
            for entity in df[selected_column].dropna().unique()[:10]:
                st.write(f"Searching for: {entity}")
                search_results[entity] = perform_web_search(entity)
            st.success("Web search completed.")
            st.json(search_results)
        except Exception as e:
            st.error(f"An error occurred during the web search: {e}")


2024-11-18 17:30:00.256 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:30:00.260 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:30:00.262 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:30:00.264 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 17:30:00.267 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [13]:

if parsed_data:
    parsed_df = pd.DataFrame.from_dict(parsed_data, orient="index", columns=["Extracted Info"])
    st.write("Extracted Data:")
    st.dataframe(parsed_df)

    # Download button for results
    st.download_button(
        label="Download Extracted Data",
        data=parsed_df.to_csv(index=False),
        file_name="extracted_data.csv",
        mime="text/csv"
    )


In [14]:
serpapi_key = "YOUR_SERPAPI_KEY"
openai_api_key = "YOUR_OPENAI_API_KEY"


testing


In [15]:
import os
import pandas as pd

# Path to dataset directory
dataset_path = "/root/.cache/kagglehub/datasets/peopledatalabssf/free-7-million-company-dataset/versions/1"

try:

    files = os.listdir(dataset_path)
    print("Files in the dataset directory:", files)


    dataset_file = None
    for file in files:
        if file.endswith(".csv"):
            dataset_file = os.path.join(dataset_path, file)
            break

    if not dataset_file:
        raise FileNotFoundError("No CSV file found in the dataset directory.")

    # Load the dataset
    df = pd.read_csv(dataset_file)
    print("Dataset loaded successfully! Here's a preview:")
    print(df.head())

except Exception as e:
    print(f"Error loading dataset: {e}")


Files in the dataset directory: ['companies_sorted.csv']
Dataset loaded successfully! Here's a preview:
   Unnamed: 0                       name         domain  year founded  \
0     5872184                        ibm        ibm.com        1911.0   
1     4425416  tata consultancy services        tcs.com        1968.0   
2       21074                  accenture  accenture.com        1989.0   
3     2309813                    us army     goarmy.com        1800.0   
4     1558607                         ey         ey.com        1989.0   

                              industry size range  \
0  information technology and services     10001+   
1  information technology and services     10001+   
2  information technology and services     10001+   
3                             military     10001+   
4                           accounting     10001+   

                                 locality         country  \
0       new york, new york, united states   united states   
1              b